In [83]:
# IMPORTACIONES
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import timedelta
from shapely import wkb

In [84]:
# CONECCION CON AMIGOCLOUD
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [85]:
# PATHS
# RUTAS PARA NOTIFICACION DE LABORES DE LIBERACION DE TRICHOGRAMMA
path_xlsx_plan = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\pd_trichogramma_2024 - copia.xlsx'

# PILA DE MENSAJES
path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

In [86]:
# DATOS DE PILOTOS
tecnico_telf = {
    'ALEJANDRO SANCHEZ' : 78448317,
    'AURELIO GARCIA' : 79051559,
    'CHRISTIAN CALIZAYA' : 63619660,
    'EDUARDO REYES' : 75016609}

In [87]:
# EJECUTAR QUERY
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [88]:
# CORVERSION DE WKB A COORS
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

# NOTIFICACIONES DE LIBERACION

In [146]:
def get_registro_notificacion():
    # get data de nuevas notificacion de liberacion
    query = 'select * from dataset_354655 where procesado=false'
    notif = ejecutar_query_sql(33457, query, 'get')
    notif = notif['data']
    # extrae el primer elemento
    return notif

In [147]:
def get_planificacion():
    df_plan = pd.read_excel(path_xlsx_plan, sheet_name='PLANIFICACION')
    df_plan = df_plan[['unidad_03', 'unidad_04', 'telefono_1', 'telefono_2', 'telefono_3']]
    return df_plan

In [ ]:
while True:
    reg_amigo = get_registro_notificacion()
    if len(reg_amigo) = 0:
        print('No existen notificaciones nuevas')
        
    reg_amigo = notif[0]
    
    # extrae cod_ca y nom_ca del registro
    cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
    nom_ca = reg_amigo['canhero'].split(' / ')[1]
    

In [140]:
df_plan = pd.read_excel(path_xlsx_plan, sheet_name='PLANIFICACION')

In [141]:
df_plan = df_plan[['unidad_03', 'unidad_04', 'telefono_1', 'telefono_2', 'telefono_3']]

In [142]:
df_plan.head(3)

,unidad_03,unidad_04,telefono_1,telefono_2,telefono_3
0,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN
1,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN
2,18358,ROJAS AMELUNGE CARLOS,77395410,0,NaN


In [93]:
# get data de nuevas notificacion de liberacion
query = 'select * from dataset_354655 where procesado=false'
notif = ejecutar_query_sql(33457, query, 'get')
notif = notif['data']

In [94]:
notif

[{'fecha_registro': '2024-09-11 20:01:10+00:00',
  'amigo_id': '9ff59ac965874f46bca0725c07435fb2',
  'id': 2,
  'piloto': 'ALEJANDRO SANCHEZ',
  'canhero': '0 / BISMARK SOCOMPI',
  'ubicacion': '0101000020E61000009AD99DA4AF954FC0B2CDD776244831C0',
  'tipo_labor': 'LIBERACION DE TRICHOGRAMMA',
  'tipo_mensaje': 'INICIO DE LABOR',
  'procesado': False}]

In [95]:
# extrae el primer elemento
reg_amigo = notif[0]

In [96]:
# extrae cod_ca y nom_ca del registro
cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
nom_ca = reg_amigo['canhero'].split(' / ')[1]

In [97]:
# filtra en la lista de planificacion el cod_ca para obtener los numeros cel.
reg = df_plan[df_plan['unidad_03']==cod_ca].iloc[0]

unidad_03                     0
unidad_04     BISMARK SOCOMPI R
telefono_1             78194371
telefono_2             62022996
telefono_3                  NaN
Name: 746, dtype: object

In [98]:
# id del registro en amigocloud
idd = reg_amigo['id']

2

In [99]:
# extrae fecha y hora de registro menos 4 horas por desfase temprotal
fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)

In [100]:
# formatea la fecha de registro
fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')

'11-09-2024'

In [101]:
# formatea la hora de registro
hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')

'16:01:10'

In [104]:
# extrae los numeros telefonicos
telefonos = [reg['telefono_1'], reg['telefono_2'], reg['telefono_3']]
# valida los numeros telefonicos
telefonos_validos = [telefono for telefono in telefonos if isinstance(telefono, int) and len(str(telefono)) == 8]

In [106]:
# extra el tipo de labor (liberacion o pulverizacion)
labor = reg_amigo['tipo_labor']

'LIBERACION DE TRICHOGRAMMA'

In [107]:
# extrae el estado de la labor (inicio o fin de labor)
estado = reg_amigo['tipo_mensaje']

'INICIO DE LABOR'

In [108]:
# extrae nombre del piloto
tecnico = reg_amigo['piloto']

'ALEJANDRO SANCHEZ'

In [109]:
# extrae y convierte la ubicacion
ubi = convertir_wkb(reg_amigo['ubicacion'])
longitud = ubi.x  # Longitud (coordenada X)
latitud = ubi.y   # Latitud (coordenada Y)

In [110]:
def insertar_msj_stack(idd, fecha, hora, cod, nombre, telf, msj):
    linea = f'{idd};{fecha};{hora};{cod};{nombre};{telf};{msj}'
    with open(path_txt_msj, 'r') as file:
        contenido = file.readlines()
    contenido.insert(0, '\n')
    contenido.insert(0, linea)
    with open(path_txt_msj, 'w') as file:
        file.writelines(contenido)
    print(linea)

In [135]:
def anular_registro_notificacion(idd):
    query = f'update dataset_354655 set procesado = true where id = {idd}'
    exe = ejecutar_query_sql(33457, query, 'post')

In [137]:
# crea el mensaje 
def crear_msj(idd, fecha, hora, cod, nombre, telfs, labor, estado, tecnico, lon, lat):
    tec_telf = tecnico_telf[tecnico]
    for i in telfs:
        msj = ''
        if estado == 'INICIO DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se esta iniciando el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        elif estado == 'FIN DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se ha finalizado el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        insertar_msj_stack(idd, fecha, hora, cod, nombre, i, msj)
        anular_registro_notificacion(idd)

In [139]:
crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)

2;11-09-2024;16:01:10;0;BISMARK SOCOMPI;78194371;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *LIBERACION DE TRICHOGRAMMA* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2818064,-63.1694227. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
2;11-09-2024;16:01:10;0;BISMARK SOCOMPI;62022996;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *LIBERACION DE TRICHOGRAMMA* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2818064,-63.1694227. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
